In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('wordle_v1.csv')
df.head()

,datetime,text,retweet_count,reply_count,like_count
0,2022-01-28 23:59:59+00:00,Wordle 223 5/6\n\n⬛⬛⬛⬛⬛\n🟨🟩⬛⬛⬛\n⬛🟩🟩⬛⬛\n⬛🟩🟩🟩🟩\n...,0,0,1
1,2022-01-28 23:59:58+00:00,Wordle 224 5/6\n\n⬜🟨⬜⬜🟨\n🟨🟩⬜🟨⬜\n⬜🟩🟩⬜🟩\n⬜🟩🟩🟩🟩\n...,0,0,0
2,2022-01-28 23:59:58+00:00,Wordle 224 4/6*\n\n⬜🟨⬜⬜🟨\n⬜🟩🟩⬜🟩\n⬜🟩🟩🟩🟩\n🟩🟩🟩🟩🟩\...,0,1,1
3,2022-01-28 23:59:58+00:00,Wordle 224 5/6\n\n⬜🟩🟩⬜⬜\n⬜🟨🟨⬜⬜\n🟨🟩🟩⬜⬜\n⬜🟩🟩⬜🟩\n...,0,0,1
4,2022-01-28 23:59:57+00:00,@Sako_506 Wordle (ES) #22 2/6\n\n⬜⬜🟩🟨🟩\n🟩🟩🟩🟩🟩\...,0,1,0
